This can read from a locally-staged hail table directory, or use one in GCS. The table must have a pre-computed `.info.VRS` field. This is computed using `tgg_methods` `vrs_annotation_batch.py`, which uses the `vrs-python` `vcf_annotation.py`

https://github.com/broadinstitute/tgg_methods/blob/master/vrs/vrs_annotation_batch.py (last checked at `a0002f02fbd5dd25487b261e94081a3daec29c64`)

This uses gnomad_methods functions `gnomad_gks` and `add_gks_vrs/va`, to return GKS structures for hail tables that are in the gnomad v3 schema, off the branch in this pull request: https://github.com/broadinstitute/gnomad_methods/pull/556

The JSON schema is in `ga4gh/va-spec` on the `gk-pilot` branch

In [1]:
import hail as hl

In [2]:
# The clingen-public bucket will only be the week of 2023-09-18
# The clingen-public-requesterpays bucket will remain available but the client will need to 
# be authenticated with a Google Cloud account with billing enabled to pay network transfer fees

# configuration for data outputs
bucket = "clingen-public-requesterpays"
bucket = "clingen-public"

# Writes inputs array as a hail table to this destination, if not None.
# This can be useful for other testing, using this hail table as input without reconstructing it
# inputs_ht_destination_url = f"gs://{bucket}/gnomad-gks-qc/gnomad-filtered-inputs.ht"
inputs_ht_destination_url = None

# Copies output annotations as newline delimited json to this url, if not None
outputs_destination_file = f"gs://{bucket}/gnomad-gks-qc/outputs.ndjson"


# ht_url can be a gs:// path, or a file:// local path

# Publicly readable, but doesn't have all gnomad variants in it
# ht_url = f"gs://{bucket}/gnomad-gks-downsampled-100k.ht"

# Can refer to a local hail table directory
ht_url = "../downsample_to_100k_full_release.ht"

# gnomad-vrs-io-finals is a private bucket, needs a service account with read access 
# ht_url = "gs://gnomad-vrs-io-finals/ht-outputs/release_0426_dmtest_v3.1.2-Full-ht-release-output-updated-schema-050523.ht"


print(ht_url)
ht = hl.read_table(ht_url)

Initializing Hail with default parameters...


../downsample_to_100k_full_release.ht


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/Users/kferrite/dev/gnomad_methods/venv/lib/python3.11/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://172.22.34.209:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.118-a4ca239602bb
LOGGING: writing to /Users/kferrite/dev/gnomad_methods/notebooks/hail-20230919-1607-0.2.118-a4ca239602bb.log


In [3]:
import gnomad
import gnomad.utils.annotations
import gnomad.resources.grch38.gnomad

# reload (re-running this cell will reload modifications to these modules on disk)
import importlib
importlib.reload(gnomad.utils.annotations)
importlib.reload(gnomad.resources.grch38.gnomad)

from gnomad.utils.annotations import add_gks_va, add_gks_vrs
from gnomad.resources.grch38.gnomad import gnomad_gks

/Users/kferrite/dev/gnomad_methods/venv/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:49: UserWarning: Schema version http://json-schema.org/draft-07/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [4]:
import subprocess
import shutil
va_spec_clone = "gnomad-gks-v1_va-spec"
va_spec_branch = "gk-pilot"
shutil.rmtree(va_spec_clone, ignore_errors=True)
p = subprocess.run(["git", "clone", "https://github.com/ga4gh/va-spec", "gnomad-gks-v1_va-spec"],
                   check=True)
p = subprocess.run(["bash", "-c",
                    f"cd {va_spec_clone} && git checkout {va_spec_branch}"],
                   check=True)

Cloning into 'gnomad-gks-v1_va-spec'...


branch 'gk-pilot' set up to track 'origin/gk-pilot'.


Switched to a new branch 'gk-pilot'


In [5]:
import json
with open(f"{va_spec_clone}/schema/cohortAlleleFreq.json") as f:
    schema = json.load(f)

In [6]:
# GnomAD 3.1.2 
# GRCh38 expressions

inputs = [
    {
        "clinvar": "422227",
        "gnomad": "16-68737366-C-T"
    },
    {
        "clinvar": "3038",
        "gnomad": "11-108248927-T-G"
    },
    {
        "clinvar": "256155",
        "gnomad": "7-107701083-T-A"
    }
]


In [7]:
# # This cell constructs an indexed hail table from the input alleles
# # so the full gnomad dataset can be rapidly filtered by left joining it onto this.
# # Using a .filter() to check membership in the input set is slow because it needs
# # to do a table scan.
# # Too slow:
# # ht_with_coords.filter(input_gnomad_expressions.contains(ht_with_coords.genomic_coordinates))

# import pandas

# input_gnomad_expressions = hl.literal([x["gnomad"] for x in inputs])
# input_terms = [x["gnomad"].split("-") for x in inputs]

# df = pandas.DataFrame(
#     {
#         "contig": [str("chr" + i[0]) for i in input_terms],
#         "position": [int(i[1]) for i in input_terms],
#         "ref": [i[2] for i in input_terms],
#         "alt": [i[3] for i in input_terms]
#     }
# )
# input_ht = hl.Table.from_pandas(df)
# input_ht = (input_ht
#     .annotate(
#         locus=hl.locus(input_ht.contig, input_ht.position, reference_genome="GRCh38"),
#         alleles=hl.array([input_ht.ref, input_ht.alt]))
#     .drop("contig", "position", "ref", "alt")
#     .key_by("locus", "alleles"))

# input_ht.show()

,
locus,alleles
locus<GRCh38>,array<str>
chr1:11130641,"[""G"",""A""]"
chr1:55051215,"[""G"",""GA""]"
chr1:216247118,"[""C"",""A""]"
chr2:47799491,"[""C"",""G""]"
chr7:107701083,"[""T"",""A""]"
chr11:108248927,"[""T"",""G""]"
chr12:102843683,"[""C"",""T""]"
chr14:23416241,"[""G"",""A""]"


In [8]:
ht_with_coords = ht.annotate(
    genomic_coordinates = hl.format("%s-%s-%s-%s",
        ht.locus.contig[3:], # Remove 'chr'
        hl.str(ht.locus.position),
        ht.alleles[0],
        ht.alleles[1]
    )
)
ht_with_coords.show()

ht_filtered = ht_with_coords

# Indexed filter using join on the indexed vcf table constructed from inputs
# To enable filtering to the set of input variants, uncomment everything below

# ht_filtered = input_ht.join(ht_with_coords)
# ht_filtered.genomic_coordinates.show()

# # Write the filtered gnomad table to storage
# if inputs_ht_destination_url:
#     ht_filtered.write(inputs_ht_destination_url, overwrite=True)

+---------------+---------------+
| locus         | alleles       |
+---------------+---------------+
| locus<GRCh38> | array<str>    |
+---------------+---------------+
| chr1:30935    | ["T","G"]     |
| chr1:104848   | ["G","T"]     |
| chr1:110766   | ["C","T"]     |
| chr1:113813   | ["A","G"]     |
| chr1:192002   | ["A","G"]     |
| chr1:264521   | ["T","C"]     |
| chr1:288975   | ["G","A"]     |
| chr1:600754   | ["C","A"]     |
| chr1:629844   | ["A","G"]     |
| chr1:633440   | ["A","ATCCC"] |
+---------------+---------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,32192,0),(2,2.74e-05,72972,1),(0,0.00e+00,7082,0),(0,0.00e+0... |
| [(0,0.00e+00,117326,0),(4,2.72e-05,146990,0),(0,0.00e+00,51878,0),(0,0.00... |
| [(1,6.64e-06,150524,0),(1,6.58e-06,152072,0),(1,1.48e-05,67432,0),(0,0.00... |
| [(1,6.69e-06,149410,0),(1,6.59e-06,151734,0),(1,1.48e-05,67394,0),(0,0.00... |
| [(4,2.89e-05,138298,0),(6,3.99e-05,150550,0),(0,0.00e+00,65272,0),(0,0.00... |
| [(1,6.58e-06,152022,0),(2,1.31e-05,152282,0),(1,1.47e-05,67994,0),(0,0.00... |
| [(1,8.07e-06,123908,0),(1,6.92e-06,144510,0),(1,1.75e-05,57246,0),(0,0.00... |
| [(0,0.00e+00,89916,0),(7,6.72e-05,104130,0),(0,0.00e+00,43854,0),(0,0.00e... |
| [(18,1.29e-04,139564,5),(21,1.42e-04,147838,6),(6,9.20e-05,65216,2),(0,0.... |
| [(2,1.43e-05,139728,0),(4,2.75e-05,145258,0),(0,0.00e+00,65786,0),(0,0.00... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [13114,2583,1702,954,10352,3548,2052,937,387,247,116,78,88,93,49,115,16,7... |
| [2573,1256,2381,2624,28411,9358,7508,4862,2716,2968,2006,1145,1394,984,63... |
| [145,82,149,111,35412,14732,9875,5527,2971,2063,1332,794,738,490,309,278,... |
| [128,35,173,135,29164,10530,8400,6016,4423,3902,3082,2125,2077,1519,908,8... |
| [539,693,1382,2004,14474,10589,9608,6903,6081,5078,3859,3096,2749,2080,14... |
| [9,10,27,31,23375,11353,8454,5962,4334,3891,3113,

In [9]:
# Parameters for gnomad_gks/get_gks
ancestry_group_short_names = gnomad.resources.grch38.gnomad.POPS["v3"]
ancestry_groups_full_name_map = gnomad.sample_qc.ancestry.POP_NAMES
gnomad_version_label = "3.1.4"

In [10]:
vrs_variants = []
records = ht_filtered.select(
    ht_filtered.freq,
    ht_filtered.popmax,
    ht_filtered.info,
    ht_filtered.genomic_coordinates,
    vrs=ht_filtered.info.vrs
)
records = records.take(5)

variant_strs = [r.genomic_coordinates for r in records]
loci = [
    hl.locus(contig=str("chr" + v0), pos=int(v1), reference_genome="GRCh38")
    for [v0, v1, *_] in 
    [v.split("-") for v in variant_strs]
]

print(f"gnomAD allele strings: {variant_strs}")
for record in records:
    print("calling add_gks_vrs on: " + record.genomic_coordinates)
    vrs_variant = add_gks_vrs(
        input_vrs=record.info.vrs,
        input_locus=record.locus
    )
    print(json.dumps(vrs_variant, indent=2))
    vrs_variants.append(vrs_variant)


gnomAD allele strings: ['1-30935-T-G', '1-104848-G-T', '1-110766-C-T', '1-113813-A-G', '1-192002-A-G']
calling add_gks_vrs on: 1-30935-T-G
{
  "_id": "ga4gh:VA.-lcHVUGOSpWXjh1b5IHf9jS_nl93bidy",
  "type": "Allele",
  "location": {
    "type": "SequenceLocation",
    "sequence_id": "ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO",
    "interval": {
      "start": {
        "type": "Number",
        "value": 30934
      },
      "end": {
        "type": "Number",
        "value": 30935
      },
      "type": "SequenceInterval"
    },
    "_id": "ga4gh:VSL.ysTBVv823ilCQX8bpIqBI_Q0fuPZUQPq"
  },
  "state": {
    "type": "LiteralSequenceExpression",
    "sequence": "G"
  }
}
calling add_gks_vrs on: 1-104848-G-T
{
  "_id": "ga4gh:VA.4_VKSReCjw69WYAexsXH3Z3mD65wpfor",
  "type": "Allele",
  "location": {
    "type": "SequenceLocation",
    "sequence_id": "ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO",
    "interval": {
      "start": {
        "type": "Number",
        "value": 104847
      },
      "

In [11]:
# This cell takes one locus present in the input table, performs VRS and VA annotation and prints it.

# For performance on larger datasets, the interval should be much larger, at least a few thousand.

ivl_0 =hl.locus_interval(loci[0].contig, loci[0].position, loci[0].position+1, reference_genome="GRCh38")
ht_locus_0 = (hl.filter_intervals(ht_filtered, [ivl_0]))

import time
t0 = time.time()
gks_annotation = gnomad_gks(
    locus_interval=ivl_0,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=True
)
t1 = time.time()
print(f"{t1-t0} seconds")


gks_annotation = gnomad_gks(
    locus_interval=ivl_0,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=False
)
t1 = time.time()
print(f"{t1-t0} seconds")


print(json.dumps(gks_annotation[0], indent=2))

2.738466262817383 seconds


9.937304258346558 seconds
{
  "locus": {
    "contig": "chr1",
    "position": 30935,
    "reference_genome": "GRCh38"
  },
  "alleles": [
    "T",
    "G"
  ],
  "gks_vrs_variant": {
    "_id": "ga4gh:VA.-lcHVUGOSpWXjh1b5IHf9jS_nl93bidy",
    "type": "Allele",
    "location": {
      "type": "SequenceLocation",
      "sequence_id": "ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO",
      "interval": {
        "start": {
          "type": "Number",
          "value": 30934
        },
        "end": {
          "type": "Number",
          "value": 30935
        },
        "type": "SequenceInterval"
      },
      "_id": "ga4gh:VSL.ysTBVv823ilCQX8bpIqBI_Q0fuPZUQPq"
    },
    "state": {
      "type": "LiteralSequenceExpression",
      "sequence": "G"
    }
  },
  "gks_va_freq": {
    "id": "gnomAD-3.1.4-chr1-30935-T-G",
    "type": "CohortAlleleFrequency",
    "label": "Overall Cohort Allele Frequency for chr1-30935-T-G",
    "derivedFrom": {
      "id": "gnomAD3.1.4",
      "type": "DataSet",

In [12]:
# This cell takes one locus present in the input table, performs VRS and VA annotation and prints it.

# For performance on larger datasets, the interval should be much larger, at least a few thousand.
print(hl.eval(loci[0].contig))
ivl_0 =hl.locus_interval(loci[0].contig, loci[0].position, loci[0].position+1, reference_genome="GRCh38")
ht_locus_0 = (hl.filter_intervals(ht_filtered, [ivl_0]))


larger_interval = hl.locus_interval(contig="chr1",
                       start=1,
                       end=int(1e7),
                       reference_genome="GRCh38")

import time
t0 = time.time()
gks_annotations = gnomad_gks(
    locus_interval=larger_interval,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=False
)
t1 = time.time()
l = len(gks_annotations)
td = t1 - t0
print(f"Annotated {l} records in {td} seconds ({td/l:.2f} sec/rec)")

print(json.dumps(gks_annotation[0], indent=2))

chr1


Annotated 303 records in 57.202012062072754 seconds (0.19 sec/rec)
{
  "locus": {
    "contig": "chr1",
    "position": 30935,
    "reference_genome": "GRCh38"
  },
  "alleles": [
    "T",
    "G"
  ],
  "gks_vrs_variant": {
    "_id": "ga4gh:VA.-lcHVUGOSpWXjh1b5IHf9jS_nl93bidy",
    "type": "Allele",
    "location": {
      "type": "SequenceLocation",
      "sequence_id": "ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO",
      "interval": {
        "start": {
          "type": "Number",
          "value": 30934
        },
        "end": {
          "type": "Number",
          "value": 30935
        },
        "type": "SequenceInterval"
      },
      "_id": "ga4gh:VSL.ysTBVv823ilCQX8bpIqBI_Q0fuPZUQPq"
    },
    "state": {
      "type": "LiteralSequenceExpression",
      "sequence": "G"
    }
  },
  "gks_va_freq": {
    "id": "gnomAD-3.1.4-chr1-30935-T-G",
    "type": "CohortAlleleFrequency",
    "label": "Overall Cohort Allele Frequency for chr1-30935-T-G",
    "derivedFrom": {
      "id"